In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import os
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plt.style.use("ggplot")

In [2]:
competition = 'asl-signs'

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('..') / 'input'  / 'competition'
    # !pip install -Uqq fastai
else:
    import zipfile, kaggle
    path = Path.home() / '.data' / 'asl-signs'
    if not path.exists():
        path.mkdir(exist_ok=True)
        kaggle.api.competition_download_cli(competition, path=path)
        zipfile.ZipFile(path / f'{competition}.zip').extractall(path)

# Labeled Data

In [3]:
with open(path / 'sign_to_prediction_index_map.json') as f:
    sign_labels = json.load(f)

In [4]:
train = (pd.read_csv(path / 'train_with_meta.csv')
         .assign(idx=lambda x: x.participant_id.astype(str) + '_' + x.sequence_id.astype(str))
         .set_index('idx')
        )
train.head()

,path,participant_id,sequence_id,sign,cnt_partial_nulls,cnt_partial_nulls_by_frame,total_frames,face,left_hand,pose,right_hand
idx,,,,,,,,,,,
26734_1000035562,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,0.0,0.0,23.0,23.0,0.0,23.0,11.0
28656_1000106739,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,0.0,0.0,11.0,11.0,0.0,11.0,2.0
16069_100015657,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,0.0,0.0,105.0,105.0,28.0,105.0,0.0
25571_1000210073,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,0.0,0.0,12.0,12.0,0.0,12.0,12.0
62590_1000240708,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,0.0,0.0,18.0,18.0,0.0,18.0,18.0


In [5]:
def interpolate_values(landmarks):
#     idx = pd.MultiIndex.from_product(
#         [frames, landmarks.landmark_index.unique().tolist()],
#         names=['frame', 'landmark_index']
#     )

    landmarks = (landmarks.set_index(['frame', 'landmark_index'])
                 #.reindex(idx, fill_value=np.nan)
                )
    landmarks = (landmarks.sort_index(level=[1, 0])
                 .assign(x=lambda x: x.groupby('landmark_index').x.apply(lambda g: g.interpolate()),
                         y=lambda x: x.groupby('landmark_index').y.apply(lambda g: g.interpolate()),
                         z=lambda x: x.groupby('landmark_index').z.apply(lambda g: g.interpolate()),
                  )
                 .sort_index(level=[0, 1])
                 .reset_index()
                )
    return landmarks

# 3D Plotly Viz

In [6]:
record_id = '61333_3563388072'
sample = train.loc[record_id, :]
pd.read_parquet(path / sample['path'])

,frame,row_id,type,landmark_index,x,y,z
0,42,42-face-0,face,0,0.492295,0.426160,-0.031124
1,42,42-face-1,face,1,0.494285,0.393884,-0.057714
2,42,42-face-2,face,2,0.487761,0.403856,-0.031419
3,42,42-face-3,face,3,0.480398,0.362385,-0.044600
4,42,42-face-4,face,4,0.495073,0.384079,-0.061128
...,...,...,...,...,...,...,...
5425,51,51-right_hand-16,right_hand,16,NaN,NaN,NaN
5426,51,51-right_hand-17,right_hand,17,NaN,NaN,NaN
5427,51,51-right_hand-18,right_hand,18,NaN,NaN,NaN
5428,51,51-right_hand-19,right_hand,19,NaN,NaN,NaN


In [7]:
class Visualizer:
    COORD_COLS = ['x', 'y', 'z']
    LANDMARK_TYPES = ['left_hand', 'pose', 'right_hand']
    
    def __init__(self, train, interpolate_values=False):
        self.train = train
        self.interpolate_values = interpolate_values
    
    def sample_by_sign(self, sign, random_state=69, mask=None):
        sign_mask = self.train.loc[:, 'sign'] == sign
        mask = sign_mask if mask is None else (sign_mask & mask)
        sample = self.train.loc[mask, :].sample(n=1, random_state=random_state)

        landmarks = self.read_parquet(sample['path'][0])
        return sample, landmarks 
        
    def select_record(self, record_id):
        sample = self.train.loc[record_id, :]
        landmarks = self.read_parquet(sample['path'])
        return sample, landmarks
    
    def read_parquet(self, pq_name):
        landmarks = pd.read_parquet(path / pq_name)
        landmarks = landmarks.loc[landmarks.type.isin(self.LANDMARK_TYPES), ]
        # landmarks = landmarks.loc[~landmarks.loc[:, self.COORD_COLS].isna().all(axis=1), :]
        pose_mask = landmarks.loc[:, 'type'] == 'pose'
        landmarks.loc[pose_mask, 'x'] = -landmarks.loc[pose_mask, 'x']
        return landmarks
    
    def visualize(self, sign=None, random_state=69, record_id=None,
                  poses=['right_hand', 'pose', 'left_hand']):
        if sign is None and record_id is None:
            raise ValueError("Must enter random state or record_id")
        elif sign is not None:
            self.sample, self.landmarks = self.sample_by_sign(sign, random_state)
        else:
            self.sample, self.landmarks = self.select_record(record_id)
            
        self.frames = self.landmarks.frame.unique().tolist()
        
        self.fig = make_subplots(rows=1, cols=len(poses),     
                                 specs=[[{"type": "scatter3d"} for i in range(len(poses))]],
                                )

        for i, pose in enumerate(poses):
            self.add_trace(pose, i+1)
        
        steps = list()
        for i, frame in enumerate(self.frames):
            step = dict(
                method='update',
                args=[{"visible": [False] * len(self.frames) * 6},
                      {"title": "Slider switched to step: " + str(frame)}],
                label=str(frame),
                value=i
            )
            step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
            for j in range(len(poses) * 2):
                step["args"][0]["visible"][i + len(self.frames) * j] = True  # Toggle j'th trace to "visible"
            steps.append(step)
        
        active = 0
        for j in range(len(poses) * 2):
            self.fig.data[active + len(self.frames) * j].visible = True  # Toggle j'th trace to "visible"

        sliders = [dict(
            active=active,
            currentvalue={'prefix': "Frame: "},
            pad={'t': 50},
            steps=steps
        )]
        
        
        self.fig.update_layout(
            sliders=sliders,
        )
        centers = self.landmarks.groupby('type')[viz.COORD_COLS].mean()
        
        pose_center = dict(self.landmarks.query('landmark_index in (11, 12, 23, 24) and type == "pose"')
                           [self.COORD_COLS].mean())
        pose_camera = dict(
            up=dict(x=0, y=-1, z=0),
            # center=pose_center,
            eye=dict(x=0, y=0, z=-2.5)
        )
        hand_camera = dict(
            up=dict(x=-0.5, y=-1, z=-0.5),
            # center=dict(x=0, y=0, z=0),
            eye=dict(x=-1.5, y=1, z=-2.5)
        )
        self.fig.layout.scene1.camera = hand_camera
        self.fig.layout.scene2.camera = pose_camera
        self.fig.layout.scene3.camera = hand_camera

        self.fig.show()
        
    def add_trace(self, landmarks_type, col):
        type_mask = self.landmarks.type == landmarks_type
        type_landmarks = self.landmarks.loc[type_mask, :]
        if self.interpolate_values:
            type_landmarks = interpolate_values(type_landmarks)
        
        # markers
        for frame in self.frames:
            frame_landmarks = type_landmarks.loc[type_landmarks.frame == frame, ]
            self.fig.add_trace(
                go.Scatter3d(
                    visible=False, 
                    x=frame_landmarks.x, y=frame_landmarks.y, z=frame_landmarks.z, text=frame_landmarks.landmark_index,
                    marker=dict(size=8, opacity=0.8),
                    mode="markers",
                    name=landmarks_type,
                ),
                row=1, col=col,
            )

        # segments
        for frame in self.frames:
            frame_landmarks = type_landmarks.loc[type_landmarks.frame == frame, ]
            landmark_lines = create_landmark_lines(frame_landmarks, landmarks_type)
            self.fig.add_trace(
                go.Scatter3d(
                    visible=False, 
                    x=landmark_lines.x, y=landmark_lines.y, z=landmark_lines.z,
                    mode="lines",
                    hoverinfo='skip',
                    name=landmarks_type
                ),
                row=1, col=col,
            )

def create_landmark_lines(frame_landmarks, landmarks_type='hand'):
    empty_row = pd.Series({
        'frame': None, 'row_id': None, 'type': None, 'x': None, 'y': None, 'z': None
    })
    frame_landmarks = frame_landmarks.set_index('landmark_index')
    
    if frame_landmarks.empty:
        line_indices = [None]
    elif landmarks_type in ['hand', 'left_hand', 'right_hand']:
        line_indices = [0, 1, None, 0, 5, None, 0, 17, None, 
                        1, 2, 3, 4, None,
                        5, 6, 7, 8, None,
                        5, 9, None,
                        9, 10, 11, 12, None,
                        9, 13, None,
                        13, 14, 15, 16, None,
                        13, 17, None,
                        17, 18, 19, 20, None
                       ]
    elif landmarks_type == "pose":
        line_indices = [0, 1, 2, 3, 7, None, 
                        0, 4, 5, 6, 7, 8, None,
                        11, 13, 15, 17, 19, 15, 21, None,
                        11, 12, 14, 16, 18, 20, 16, 22, None,
                        11, 23, 25, 27, 31, 29, 27, None,
                        12, 24, 26, 28, 30, 32, 28, None,
                        23, 24, None
               ]
    else:
        raise ValueError()
    lines = pd.DataFrame([frame_landmarks.loc[i, ] if i is not None else empty_row
                          for i in line_indices
                         ])
    return lines

In [9]:
viz = Visualizer(train, True)
viz.visualize(sign='thankyou', random_state=69699)